In [75]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [76]:
class Subproblem_Quad_IP():
    def __init__(self, obj_coeff, constraints_coeff):
        self.n_dim = len(obj_coeff)
        self.obj_coeff = obj_coeff
        self.constraints_coeff = constraints_coeff
        
    def compute_obj(self, lamd):
        self.relaxed_obj_coeff = []
        for i in range(self.n_dim):
            self.relaxed_obj_coeff.append({'quad': obj_coeff[i], 'linear': (lamd * constraints_coeff[:,i]).sum()})
            
    def solve(self):
        self.opt_solution = np.zeros(self.n_dim)
        for i in range(self.n_dim):
            coeff = self.relaxed_obj_coeff[i]
            self.opt_solution[i] = round(-1 * coeff['linear']/ (2 *coeff['quad']))
        self.opt_solution = np.maximum(0, self.opt_solution)
    
    def solve_surrogate(self, dual_problem):
        self.compute_costfun(dual_problem)
        for i in range(self.n_dim):
            coeff = self.relaxed_obj_coeff[i]
            self.opt_solution[i] = round(-1 * coeff['linear']/ (2 *coeff['quad']))
            new_obj_value 
            if 
    
    def compute_costfun(self, dual_problem):
        costfun_orig = np.dot((self.opt_solution * self.opt_solution).T, obj_coeff)
        cost_constraints = 0
        for i in range(len(dual_problem_qip.lamd)):
            cost_constraints += self.lamd[i] * (np.dot(self.opt_solution.T, self.constraints_coeff[i,:]) - rhs[i])
        return costfun_orig + cost_constraints

In [77]:
class Dual_Problem():
    def __init__(self, n_constraints):
        self.lamd = np.zeros(n_constraints)
        self.m, self.r, self.alpha = 10, 0.01, 1.0
        self.step_init, self.step, self.iteration_time = 0.0, 0.0, 0
        self.subgradients_init = np.zeros(n_constraints)
        self.subgradients = np.zeros(n_constraints)
        self.subg_norm_iterations, self.step_iterations, self.costfun_iterations = [], [], []
    
    def compute_subgradients(self, subproblem_qip):
        for i in range(len(self.lamd)):
            self.subgradients[i] = np.dot(subproblem_qip.opt_solution.T, subproblem_qip.constraints_coeff[i,:]) - rhs[i]
        if self.iteration_time == 0:
            self.subgradients_init = self.subgradients.copy()
                 
    def compute_stepsize(self, subproblem_qip):
        if self.iteration_time == 0:
            self.step_init = (417 - self.compute_costfun(subproblem_qip))/ np.linalg.norm(self.subgradients_init)**2
        else:
            p = 1 - 1/(self.iteration_time ** self.r)
            self.alpha = self.alpha * (1 - 1/(self.m * self.iteration_time ** p))
            if np.linalg.norm(self.subgradients) != 0:
                self.step = self.alpha * self.step_init * np.linalg.norm(self.subgradients_init) / np.linalg.norm(self.subgradients)
            else:
                self.step = 0
                print("subgradients = 0")
        self.step_iterations.append(self.step)
        
    def compute_stepsize_simple(self, subproblem_qip):
        self.step = (417 - self.compute_costfun(subproblem_qip))/ np.linalg.norm(self.subgradients)**2
        self.step_iterations.append(self.step)
        if self.iteration_time == 0:
            self.step_init = self.step
        
    def update_lamd(self):
        if self.iteration_time == 0:
            self.lamd = self.lamd + self.step_init * self.subgradients_init
        else:
            self.lamd = self.lamd + self.step * self.subgradients
        self.lamd = np.maximum(self.lamd, 0)
        self.iteration_time += 1
        
    def compute_costfun(self, subproblem_qip):
        costfun_orig = np.dot((subproblem_qip.opt_solution * subproblem_qip.opt_solution).T, obj_coeff)
        cost_constraints = 0
        for i in range(len(self.lamd)):
            cost_constraints += self.lamd[i] * (np.dot(subproblem_qip.opt_solution.T, subproblem_qip.constraints_coeff[i,:]) - rhs[i])
        return costfun_orig + cost_constraints       

In [78]:
n_constraints = 2
lamd = np.zeros(n_constraints)
obj_coeff = np.array([0.5, 0.1, 0.5, 0.1, 0.5, 0.1])
constraints_coeff = np.array([[-1, 0.2, -1, 0.2, -1, 0.2], [-5, 1, -5, 1, -5, 1]])
rhs = [-48, -250]
subproblem_qip = Subproblem_Quad_IP(obj_coeff, constraints_coeff)
subproblem_qip.compute_obj(lamd)
print("the dimension of decision variables:", subproblem_qip.n_dim)
print("coefficients of objective function:", subproblem_qip.obj_coeff)
print("coefficients of constraints:", subproblem_qip.constraints_coeff)
print("relaxed problem objective coefficients:", subproblem_qip.relaxed_obj_coeff)
subproblem_qip.solve()
print("optimal solution:", subproblem_qip.opt_solution)

the dimension of decision variables: 6
coefficients of objective function: [0.5 0.1 0.5 0.1 0.5 0.1]
coefficients of constraints: [[-1.   0.2 -1.   0.2 -1.   0.2]
 [-5.   1.  -5.   1.  -5.   1. ]]
relaxed problem objective coefficients: [{'quad': 0.5, 'linear': 0.0}, {'quad': 0.1, 'linear': 0.0}, {'quad': 0.5, 'linear': 0.0}, {'quad': 0.1, 'linear': 0.0}, {'quad': 0.5, 'linear': 0.0}, {'quad': 0.1, 'linear': 0.0}]
optimal solution: [0. 0. 0. 0. 0. 0.]


In [79]:
dual_problem_qip = Dual_Problem(n_constraints)
dual_problem_qip.compute_subgradients(subproblem_qip)
print("subgradients = ",dual_problem_qip.subgradients_init)
dual_problem_qip.compute_stepsize(subproblem_qip)
print("step size = ", dual_problem_qip.step)
print("the cost function of dual problem:", dual_problem_qip.compute_costfun(subproblem_qip))
dual_problem_qip.update_lamd()
print("lamd = ", dual_problem_qip.lamd)

subgradients =  [ 48. 250.]
step size =  0.0
the cost function of dual problem: 0.0
lamd =  [0.30886982 1.60869699]


In [81]:
max_itertimes = 200
n_constraints = 2
lamd = np.zeros(n_constraints)
obj_coeff = np.array([0.5, 0.1, 0.5, 0.1, 0.5, 0.1])
constraints_coeff = np.array([[-1, 0.2, -1, 0.2, -1, 0.2], [-5, 1, -5, 1, -5, 1]])
rhs = [-48, -250]
subproblem_qip = Subproblem_Quad_IP(obj_coeff, constraints_coeff)
dual_problem_qip = Dual_Problem(n_constraints)
for i in range(max_itertimes):
    subproblem_qip.compute_obj(dual_problem_qip.lamd)
    subproblem_qip.solve()
    print("iteration times:", i)
    print("optimal solution:", subproblem_qip.opt_solution)
    dual_problem_qip.compute_subgradients(subproblem_qip)
    print("subgradients:", dual_problem_qip.subgradients)
    dual_problem_qip.compute_stepsize(subproblem_qip)
    dual_problem_qip.update_lamd()
    print("lamd: ", dual_problem_qip.lamd)
    print("step:", dual_problem_qip.step, "  dual problem cost function:", dual_problem_qip.compute_costfun(subproblem_qip))
print("optimal solution = ", subproblem_qip.opt_solution)
print("subgradients = ", dual_problem_qip.subgradients)

iteration times: 0
optimal solution: [0. 0. 0. 0. 0. 0.]
subgradients: [ 48. 250.]
lamd:  [0.30886982 1.60869699]
step: 0.0   dual problem cost function: 416.99999999999994
iteration times: 1
optimal solution: [8. 0. 8. 0. 8. 0.]
subgradients: [ 24. 130.]
lamd:  [0.57652021 3.05846994]
step: 0.011152099606251643   dual problem cost function: 507.4375776889173
iteration times: 2
optimal solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.57652021 4.38601905]
step: 0.13275491057053726   dual problem cost function: 427.860190485308
iteration times: 3
optimal solution: [23.  0. 23.  0. 23.  0.]
subgradients: [-21. -95.]
lamd:  [0.3182915  3.21784154]
step: 0.012296605353891887   dual problem cost function: 481.1209321881044
iteration times: 4
optimal solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.3182915 4.2968436]
step: 0.10790020647193525   dual problem cost function: 426.96843604630425
iteration times: 5
optimal solution: [22.  0. 22.  0. 22.  0.

step: 1.0702530192286128e-05   dual problem cost function: 417.0003472751041
iteration times: 107
optimal solution: [17.  0. 17.  0. 17.  0.]
subgradients: [-3. -5.]
lamd:  [0.         3.29995037]
step: 1.6871832209281923e-05   dual problem cost function: 417.00024815825316
iteration times: 108
optimal solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.         3.30004081]
step: 9.043759641442131e-06   dual problem cost function: 417.00040805945775
iteration times: 109
optimal solution: [17.  0. 17.  0. 17.  0.]
subgradients: [-3. -5.]
lamd:  [0.         3.29996951]
step: 1.425898641104007e-05   dual problem cost function: 417.00015244493136
iteration times: 110
optimal solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.         3.30004595]
step: 7.644318143934064e-06   dual problem cost function: 417.0004595419516
iteration times: 111
optimal solution: [17.  0. 17.  0. 17.  0.]
subgradients: [-3. -5.]
lamd:  [0.         3.29998568]
step: 1.2054278

step: 1.2123855915060938e-08   dual problem cost function: 417.0000003161983
iteration times: 190
optimal solution: [17.  0. 17.  0. 17.  0.]
subgradients: [-3. -5.]
lamd:  [0.         3.29999994]
step: 1.9202167658801738e-08   dual problem cost function: 417.00000032195504
iteration times: 191
optimal solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.         3.30000004]
step: 1.0340885227341783e-08   dual problem cost function: 417.00000039017846
iteration times: 192
optimal solution: [17.  0. 17.  0. 17.  0.]
subgradients: [-3. -5.]
lamd:  [0.         3.29999996]
step: 1.637967280064096e-08   dual problem cost function: 417.0000002144026
iteration times: 193
optimal solution: [16.  0. 16.  0. 16.  0.]
subgradients: [ 0. 10.]
lamd:  [0.         3.30000005]
step: 8.821664002339027e-09   dual problem cost function: 417.0000004533612
iteration times: 194
optimal solution: [17.  0. 17.  0. 17.  0.]
subgradients: [-3. -5.]
lamd:  [0.         3.29999998]
step: 1.3974480